# Курсовая работа "Построение моделей расписания автобусов и сравнение их эффективностей"


Выполнил студент группы БВТ2203 Зотов Иван


## Практическая реализация


### 1. Импорт и загрузка библиотек


In [137]:
import random
from dataclasses import dataclass
from datetime import datetime, time, timedelta

### 2. Вспомогательные функции и параметры

In [138]:
random.seed(42)

In [139]:
DAYS_OF_WEEK = [
    "Monday",
    "Tuesday",
    "Wednesday",
    "Thursday",
    "Friday",
    "Saturday",
    "Sunday",
]

In [140]:
def get_monday_datetime():
    time = datetime.now()
    days_ahead = (7 - time.weekday()) % 7
    return time + timedelta(days=days_ahead)

### 3. Основные параметры


#### 3.1. Параметры времени

Внимание!!! 

Во всей программе используется следующая терминология для обозначения временных промежутков:
- промежутки, помеченные как bounds, обозначают *полуинтервал* вида [a;b): от начального элемента включительно и до конечного элемента не включительно;
- промежутки, помеченные как interval, обозначают *отрезок* вида \[a;b\]: от включая оба элемента.



In [141]:
@dataclass(frozen=True)
class Times:
    DRIVER_A_WT_LIMIT = 8  # WT - work time, время работы
    DRIVER_A_BREAK_LEN = 60

    DRIVER_A_WORK_START_HOURS_BOUNDS = (6, 11)
    DRIVER_A_DAYS_OFF = ["Saturday", "Sunday"]
    DRIVER_A_BREAK_START_HOURS_BOUNDS = (13, 15)

    DRIVER_B_WT_LIMIT = 12
    DRIVER_B_BREAK_LEN = 20

    DRIVER_B_DAY_OFF_PERIOD = 2
    DRIVER_B_BREAK_START_INTERVAL = (2, 4)

    SHIFT_CHANGE_LEN = 15

    ROUTE_TOTAL_TIME = 48
    ROUTE_TOTAL_TIME_ERROR_DEFAULT = 10
    ROUTE_TOTAL_TIME_ERROR_PEAK = 5

    MORNING_PEAK_HOURS_BOUNDS = (7, 10)
    EVENING_PEAK_HOURS_BOUNDS = (17, 20)
    NIGHT_HOURS_BOUNDS = (22, 6)


TIMES = Times()

#### 3.2. Основные параметры главных сущностей

In [142]:
DRIVERS_MAX_AMOUNT = 10
DRIVERS_A_AMOUNT = 6
DRIVER_A_TYPE = "DRIVER_A"
DRIVERS_B_AMOUNT = DRIVERS_MAX_AMOUNT - DRIVERS_A_AMOUNT
DRIVER_B_TYPE = "DRIVER_B"

BUSES_INIT_AMOUNT = 8
BUSES_A_AMOUNT = 6
BUS_A_TYPE = "BUS_A"
BUS_A_CAPACITY = 120
BUSES_B_AMOUNT = BUSES_INIT_AMOUNT - BUSES_A_AMOUNT
BUS_B_TYPE = "BUS_B"
BUS_B_CAPACITY = 80

#### 3.3. Экономические параметры

In [143]:
DRIVER_A_SALARY = 46
DRIVER_B_SALARY = 40

BUS_A_MAINTENANCE = 140
BUS_B_MAINTENANCE = 120

BUS_TICKET_PRICE = 35

#### 3.4. Параметры маршрута

In [144]:
BUS_STATIONS_AMOUNT = 15
ROUTE_DIRECTION_TYPE_FORWARD = "FORWARD"
ROUTE_DIRECTION_TYPE_BACKWARD = "BACKWARD"

#### 3.5. Параметр пассажиропотока

In [145]:
def generate_passenger_flow(weekday=True):
    flow = []

    for hour in range(24):
        if (
            TIMES.MORNING_PEAK_HOURS_BOUNDS[0]
            <= hour
            < TIMES.MORNING_PEAK_HOURS_BOUNDS[1]
        ):
            flow.append((30, 50) if weekday else (10, 20))
        elif (
            TIMES.EVENING_PEAK_HOURS_BOUNDS[0]
            <= hour
            < TIMES.EVENING_PEAK_HOURS_BOUNDS[1]
        ):
            flow.append((40, 60) if weekday else (15, 25))
        elif TIMES.NIGHT_HOURS_BOUNDS[0] <= hour or hour < TIMES.NIGHT_HOURS_BOUNDS[1]:
            flow.append((0, 5) if weekday else (0, 3))
        else:
            flow.append((10, 20) if weekday else (5, 15))

    return flow


PASSENGER_FLOW = [
    generate_passenger_flow(weekday=True),
    generate_passenger_flow(weekday=True),
    generate_passenger_flow(weekday=True),
    generate_passenger_flow(weekday=True),
    generate_passenger_flow(weekday=True),
    generate_passenger_flow(weekday=False),
    generate_passenger_flow(weekday=False),
]

### 4. Общие модели

#### 4.1. Действительные сущности

In [146]:
class Driver:
    work_start_hour: int
    work_end_hour: int

    def __init__(self, id: int, type):
        self.id = id
        self.name = f"Driver №{self.id}"

        self.type = type

        self.is_active = False
        self.is_in_the_end = False
        self.is_on_break = False
        self.is_on_shift_change = False
        self.works_today = False
        self.has_worked_out_today = False
        self.current_work_time = 0

    @property
    def is_available(self):
        return (
            not self.is_active
            and not self.is_on_break
            and not self.is_on_shift_change
            and self.works_today
            and not self.has_worked_out_today
        )
    
    def can_finish_work_today(self, timer):
        if timer.is_currently_peak_hour():
            if self.type == DRIVER_A_TYPE:
                return (
                    self.current_work_time // 60 >= TIMES.DRIVER_A_WT_LIMIT or
                    self.current_work_time + TIMES.ROUTE_TOTAL_TIME + TIMES.ROUTE_TOTAL_TIME_ERROR_PEAK >= TIMES.DRIVER_A_WT_LIMIT
                )
            elif self.type == DRIVER_B_TYPE:
                return (
                    self.current_work_time // 60 >= TIMES.DRIVER_B_WT_LIMIT or
                    self.current_work_time + TIMES.ROUTE_TOTAL_TIME + TIMES.ROUTE_TOTAL_TIME_ERROR_PEAK >= TIMES.DRIVER_A_WT_LIMIT
                )
        else:
            if self.type == DRIVER_A_TYPE:
                return (
                    self.current_work_time // 60 >= TIMES.DRIVER_A_WT_LIMIT or
                    self.current_work_time + TIMES.ROUTE_TOTAL_TIME +
                    TIMES.ROUTE_TOTAL_TIME_ERROR_DEFAULT >= TIMES.DRIVER_B_WT_LIMIT
                )
            elif self.type == DRIVER_B_TYPE:
                return (
                    self.current_work_time // 60 >= TIMES.DRIVER_B_WT_LIMIT or
                    self.current_work_time + TIMES.ROUTE_TOTAL_TIME +
                    TIMES.ROUTE_TOTAL_TIME_ERROR_DEFAULT >= TIMES.DRIVER_B_WT_LIMIT
                )
        return False

In [147]:
class DriverA(Driver):
    def __init__(self, id: int):
        super().__init__(id, type=DRIVER_A_TYPE)

        self.has_gone_for_the_break_today = False

    def reset(self):
        self.is_active = False
        self.is_in_the_end = False
        self.is_on_break = False
        self.is_on_shift_change = False
        self.is_working_today = False
        self.has_worked_out_today = False
        self.current_work_time = 0

        self.has_gone_for_the_break_today = False

        self.work_start_hour = None
        self.work_end_hour = None

In [148]:
class DriverB(Driver):
    def __init__(self, id: int):
        super().__init__(id, type=DRIVER_B_TYPE)

        self.mins_no_break = 0
        self.days_off_current_days = None

    def reset(self):
        self.is_active = False
        self.is_on_break = False
        self.is_on_shift_change = False
        self.is_working_today = False
        self.has_worked_out_today = False
        self.current_work_time = 0

        self.mins_no_break = 0
        self.days_off_current_days = None

        self.work_start_hour = None
        self.work_end_hour = None

In [149]:
class BusStation:
    def __init__(self, id: int):
        self.id = id
        self.name = f"BusStation №{self.id}"

        self.passengers = 0

    def spawn_passengers(self, timer):
        hour = timer.current_value.hour
        pass_min = PASSENGER_FLOW[hour][0]
        pass_max = PASSENGER_FLOW[hour][1]

        self.passengers = int(random.uniform(pass_min, pass_max) / timer.step)

    def bring_passengers(self, passengers_amount):
        self.passengers -= passengers_amount
        return passengers_amount

    def bring_all_passengers(self):
        return self.bring_passengers(self.passengers)

    def get_stop_duration(self, timer):
        if timer.is_currently_peak_hour():
            return random.randint(0, 2)
        elif timer.is_currently_night_time():
            return 0
        else:
            return random.randint(0, 1)

In [150]:
class Bus:
    def __init__(self, id, type, capacity: int, driver: Driver | None):
        self.id = id
        self.name = f"Bus №{self.id}"

        self.type = type

        self.capacity = capacity
        self.driver = driver

        self.is_active = False
        self.current_work_time = 0
        self.current_passengers = 0
        
        self.route_direction = None
        self.last_bus_station_index = None
        self.current_mins_no_station = None
        
        self.is_on_station = False
        self.is_on_depot = False
        self.current_mins_stopped = None
        self.stop_duration = None

        self.total_ride_passengers = 0

    def try_to_take_passengers(self, bus_station: BusStation):
        if self.current_passengers + bus_station.passengers <= self.capacity:
            new_passengers = bus_station.bring_all_passengers()
            self.current_passengers += new_passengers
            self.total_ride_passengers += new_passengers
        elif (
            self.current_passengers + bus_station.passengers <= self.capacity
            and self.current_passengers < self.capacity
        ):
            new_passengers = bus_station.bring_passengers(
                self.capacity - self.current_passengers
            )
            self.current_passengers += new_passengers
            self.total_ride_passengers += new_passengers

    def drop_off_passengers(self):
        num_to_leave = max(1, self.current_passengers // 5)
        self.current_passengers -= random.randint(0, num_to_leave)
        
    def drop_off_all_passengers(self):
        self.current_passengers = 0

    def reset(self):
        self.is_active = False
        self.current_work_time = 0
        self.current_passengers = 0
        
        self.route_direction = None
        self.last_bus_station_index = None
        self.current_mins_no_station = None
        
        self.is_on_station = False
        self.is_on_depot = False
        self.current_mins_stopped = None
        self.stop_duration = None
        
        self.total_ride_passengers = 0

In [151]:
class BusA(Bus):
    def __init__(self, id, driver):
        super().__init__(id, type=BUS_A_TYPE, capacity=BUS_A_CAPACITY, driver=driver)

In [152]:
class BusB(Bus):
    def __init__(self, id, driver):
        super().__init__(id, type=BUS_B_TYPE, capacity=BUS_B_CAPACITY, driver=driver)

In [153]:
class BusDepot:
    def __init__(self, id, name):
        self.id = id
        self.name = name

        self.drivers: list[Driver] = []
        self.buses: list[Bus] = []

        self.active_drivers: list[Driver] = []
        self.active_buses: list[Bus] = []

    def add_driver(self, driver: Driver):
        self.drivers.append(driver)

    def contains_driver(self, driver):
        return driver in self.drivers

    def remove_driver_if_exists(self, driver: Driver):
        if self.contains_bus(driver):
            self.drivers.remove(driver)

    def get_available_driver(self, additional_check=None):
        for driver in self.drivers:
            if driver.is_available and (
                additional_check is None or additional_check(driver)
            ):
                return driver
        return None

    def get_available_drivers(self, additional_check=None):
        return [
            driver
            for driver in self.drivers
            if driver.is_available
            and (additional_check is None or additional_check(driver))
        ]

    def add_bus(self, bus: Bus):
        self.buses.append(bus)

    def contains_bus(self, bus):
        return bus in self.buses

    def remove_bus_if_exists(self, bus: Bus):
        if self.contains_bus(bus):
            self.buses.remove(bus)

    def get_available_bus(self, additional_check=None):
        for bus in self.buses:
            if not bus.is_active:
                if additional_check is None or additional_check(bus):
                    return bus
        return None

    def get_available_buses(self):
        return [b for b in self.buses if not b.is_active]

    def add_active_driver(self, driver: Driver):
        driver.is_active = True

        self.active_drivers.append(driver)

    def contains_active_driver(self, driver):
        return driver in self.active_drivers

    def remove_active_driver_if_exists(self, driver: Driver):
        if self.contains_active_bus(driver):
            self.active_drivers.remove(driver)

    def add_active_bus(self, bus: Bus):
        bus.is_active = True

        self.active_buses.append(bus)

    def contains_active_bus(self, bus):
        return bus in self.active_buses

    def remove_active_bus_if_exists(self, bus: Bus):
        if self.contains_active_bus(bus):
            self.active_buses.remove(bus)

#### 4.2. Абстрактные сущности

##### 4.2.1. Таймер

Класс для прогона симуляции, который отвечает за ход времени

In [154]:
class Timer:
    MORNING_PEAK_HOURS_BOUNDS = TIMES.MORNING_PEAK_HOURS_BOUNDS
    EVENING_PEAK_HOURS_BOUNDS = TIMES.EVENING_PEAK_HOURS_BOUNDS
    NIGHT_HOURS_BOUNDS = TIMES.NIGHT_HOURS_BOUNDS
    
    DRIVER_A_BREAK_START_HOURS_BOUNDS = (13, 15)

    DRIVER_B_WT_LIMIT = 12
    DRIVER_B_BREAK_LEN = 10

    DRIVER_B_DAY_OFF_PERIOD = 2
    DRIVER_B_BREAK_START_INTERVAL = (2, 4)

    def __init__(self, start_value, end_value):
        self.start_value = start_value
        self.end_value = end_value
        self.step = timedelta(minutes=1)

        self.current_value = start_value

    def update_current_value(self):
        self.current_value += self.step

    def is_peak_hour(self, check_time):
        return (
            self.MORNING_PEAK_HOURS_BOUNDS[0] <= check_time.hour < self.MORNING_PEAK_HOURS_BOUNDS[1]
        ) or (
            self.EVENING_PEAK_HOURS_BOUNDS[0] <= check_time.hour < self.EVENING_PEAK_HOURS_BOUNDS[1]
        )
    
    def is_lunch_time(self, check_time):
        return (
            self.DRIVER_A_BREAK_START_HOURS_BOUNDS[0] <= check_time.hour
            or check_time.hour < self.DRIVER_A_BREAK_START_HOURS_BOUNDS[1]
        )

    def is_night_time(self, check_time):
        return (
            self.NIGHT_HOURS_BOUNDS[0] <= check_time.hour
            or check_time.hour < self.NIGHT_HOURS_BOUNDS[1]
        )

    def is_currently_peak_hour(self):
        return self.is_peak_hour(self.current_value)
    
    def is_currently_lunch_time(self):
        return self.is_lunch_time(self.current_value)

    def is_currently_night_time(self):
        return self.is_night_time(self.current_value)
    
    def is_currently_rest_day(self):
        current_day_of_week = self.current_value.weekday()
        return current_day_of_week % 3 == 1
    
    def is_currently_weekend(self):
        current_day_of_week = self.current_value.weekday()
        return current_day_of_week in [5, 6]  

##### 4.2.2. Менеджер маршрута

Класс, предназначенный для работы с маршрутом
- метод `generate_path_time_durations` генерирует массив случайных значений длительности пути между остановками, сумма которых равна total_time.

Несколько моментов, которые сразу могут быть непонятны или незаметны:
- "error" подразумевается в значение "погрешность"
- при том этом массив меньше количества остановок на одну единицу.

In [155]:
class RouteManager:
    TOTAL_TIME = TIMES.ROUTE_TOTAL_TIME
    TOTAL_TIME_ERROR_DEFAULT = TIMES.ROUTE_TOTAL_TIME_ERROR_DEFAULT
    TOTAL_TIME_ERROR_PEAK = TIMES.ROUTE_TOTAL_TIME_ERROR_PEAK

    BUS_STATIONS_AMOUNT = BUS_STATIONS_AMOUNT

    def __init__(self):
        self.bus_stations = [BusStation(i) for i in range(1, BUS_STATIONS_AMOUNT + 1)]
        self.depot_start = BusDepot(id=1, name="Start Depot")
        self.depot_end = BusDepot(id=2, name="End Depot")
        
        self.drivers: list[Driver] = []
        self.buses: list[Bus] = []
        self.active_drivers: list[Driver] = []
        self.active_buses: list[Bus] = []

        self.time_durations = self.generate_durations()

    def generate_durations(self, min_time=1, max_time=15):
        durations = [min_time] * (self.BUS_STATIONS_AMOUNT - 1)
        remaining_time = self.TOTAL_TIME - sum(durations)

        while remaining_time > 0:
            index = random.randint(0, self.BUS_STATIONS_AMOUNT - 2)
            increment = min(remaining_time, max_time - durations[index])
            durations[index] += increment
            remaining_time -= increment

        random.shuffle(durations)

        return durations
    
    def is_bus_on_boundaries(self, bus: Bus):
        index = bus.last_bus_station_index
        return index == 1 or index == BUS_STATIONS_AMOUNT
    
    def is_bus_in_the_beginning(self, bus: Bus):
        index = bus.last_bus_station_index
        return (
            bus.route_direction == ROUTE_DIRECTION_TYPE_FORWARD and index == 1
        ) or (
            bus.route_direction == ROUTE_DIRECTION_TYPE_BACKWARD and index == BUS_STATIONS_AMOUNT
        )
    
    def is_bus_in_the_end(self, bus: Bus):
        index = bus.last_bus_station_index
        return (
            bus.route_direction == ROUTE_DIRECTION_TYPE_FORWARD and index == BUS_STATIONS_AMOUNT 
        ) or (
            bus.route_direction == ROUTE_DIRECTION_TYPE_BACKWARD and index == 1
        )
    
    def has_bus_arrived_the_station(self, bus: Bus):
        return (
            bus.route_direction == ROUTE_DIRECTION_TYPE_FORWARD 
            and bus.current_mins_no_station >= self.time_durations[bus.last_bus_station_index-1]
        ) or (
            bus.route_direction == ROUTE_DIRECTION_TYPE_BACKWARD 
            and bus.current_mins_no_station >= self.time_durations[bus.last_bus_station_index-2]
        )
    
    def add_driver_to_start(self, driver: Driver):
        self.drivers.append(driver)
        self.depot_start.add_driver(driver)

    def add_driver_to_end(self, driver: Driver):
        self.drivers.append(driver)
        self.depot_end.add_driver(driver)

    def add_bus_to_start(self, bus: Bus):
        self.buses.append(bus)
        self.depot_start.add_bus(bus)

    def add_bus_to_end(self, bus: Bus):
        self.buses.append(bus)
        self.depot_end.add_bus(bus)

    def add_active_driver_to_start(self, driver: Driver):
        driver.is_active = True
        
        self.active_drivers.append(driver)
        self.depot_start.add_active_driver(driver)

    def add_active_driver_to_end(self, driver: Driver):
        driver.is_active = True
        
        self.active_drivers.append(driver)
        self.depot_end.add_active_driver(driver)

    def add_active_bus_to_start(self, bus: Bus):
        bus.is_active = True
        bus.is_on_station = True
        bus.last_bus_station_index = 1
        bus.route_direction = ROUTE_DIRECTION_TYPE_FORWARD
        
        self.active_buses.append(bus)
        self.depot_start.add_active_bus(bus)

    def add_active_bus_to_end(self, bus: Bus):
        bus.is_active = True
        bus.is_on_station = True
        bus.last_bus_station_index = BUS_STATIONS_AMOUNT
        bus.route_direction = ROUTE_DIRECTION_TYPE_BACKWARD
        
        self.active_buses.append(bus)
        self.depot_end.add_active_bus(bus)

    def reset(self):
        self.last_bus_station_index = None
        self.direction = None

##### 4.2.3. Менеджер денег

Класс для работы с экономической составляющей программы

In [156]:
class MoneyManager:
    DRIVER_A_SALARY = DRIVER_A_SALARY
    DRIVER_B_SALARY = DRIVER_B_SALARY

    BUS_A_MAINTENANCE = BUS_A_MAINTENANCE
    BUS_B_MAINTENANCE = BUS_B_MAINTENANCE

    BUS_TICKET_PRICE = BUS_TICKET_PRICE

    def __init__(self):
        self.income = 0
        self.expenses = 0
        self.profit = 0

    def calculate_profit(self):
        self.profit = self.income - self.expenses

    def pay_salary_to_one_driver(self, driver: Driver):
        if driver.type == DRIVER_A_TYPE:
            self.expenses += self.DRIVER_A_SALARY
        elif driver.type == DRIVER_B_TYPE:
            self.expenses += self.DRIVER_B_SALARY

    def pay_for_maintenance_of_one_bus(self, bus: Bus):
        if bus.type == BUS_A_TYPE:
            self.expenses += self.BUS_A_MAINTENANCE
        elif bus.type == BUS_B_TYPE:
            self.expenses += self.BUS_B_MAINTENANCE

    def earn_income_of_bus(self, bus: Bus):
        self.income += bus.total_ride_passengers * self.BUS_TICKET_PRICE

    def reset(self):
        self.income = 0
        self.expenses = 0
        self.profit = 0

##### 4.2.4. Всевозможные таблицы расписаний и их менеджер

Структуры данных таблиц расписаний и класс, предназначенный для их отображения

In [157]:
@dataclass
class BusesScheduleTable:
    time: str
    station_name: str
    bus: str
    driver: str


@dataclass
class BusStationsScheduleTable:
    station_name: str
    arrival_time: str


@dataclass
class DriversScheduleTable:
    driver: int
    start_time: str
    end_time: str
    bus: int


@dataclass
class BreaksScheduleTable:
    driver: str
    start_time: str
    end_time: str
    type: str
    bus: str


@dataclass
class ShiftChangesScheduleTable:
    bus: int
    start_time: str
    old_driver: str
    new_driver: str
    bus_station: str

In [158]:
class ScheduleManager:
    def __init__(self):
        self.buses_schedule: list[BusesScheduleTable] = []
        self.bus_stations_schedule: list[BusStationsScheduleTable] = []
        self.drivers_schedule: list[DriversScheduleTable] = []
        self.breaks_schedule: list[BreaksScheduleTable] = []
        self.shift_changes_schedule: list[ShiftChangesScheduleTable] = []

    def collect_all(self):
        return [
            self.buses_schedule,
            self.bus_stations_schedule,
            self.drivers_schedule,
            self.breaks_schedule,
            self.shift_changes_schedule
        ]

    def display_buses_schedule(self):
        pass

    def display_bus_stations_schedule(self):
        pass

    def display_drivers_schedule(self):
        pass

    def display_breaks_schedule(self):
        pass

    def display_shift_change_schedule(self):
        pass
    
    def reset(self):
        self.buses_schedule = []
        self.bus_stations_schedule = []
        self.drivers_schedule = []
        self.breaks_schedule = []
        self.shift_changes_schedule = []

### 5. Основная логика


#### 5.1. Прогонщик алгоритма

Родительский для двух подходов решения алгоритма класс, чьи свойства и методы предназначены 
для удобного прогона симуляции расписания вне зависимости от выбранного подхода. 

In [159]:
class AlgorithmRunner:
    def __init__(
        self, timer: Timer, route_manager: RouteManager, money_manager: MoneyManager, schedule_manager: ScheduleManager
    ):
        self.timer = timer
        self.rm = route_manager
        self.mm = money_manager
        self.sm = schedule_manager

        self.total_profit = 0

#### 5.2. Конфигуратор ПА

Класс, предназначенный для заполнения параметров, данных алгоритма (добавить на места автобусы, водителей и т.д.)

In [160]:
class ARConfigurator:
    def __init__(self, route_manager: RouteManager):
        self.rm = route_manager
    
    def configure_drivers(self):
        drivers_a = [DriverA(i) for i in range(1, DRIVERS_A_AMOUNT + 1)]
        drivers_b = [DriverB(i) for i in range(
            DRIVERS_A_AMOUNT + 1, DRIVERS_MAX_AMOUNT + 1)]
        
        self.distribute_items(
            drivers_a, self.rm.add_driver_to_start, self.rm.add_driver_to_end)
        self.distribute_items(
            drivers_b, self.rm.add_driver_to_start, self.rm.add_driver_to_end)

    def configure_buses(self):
        buses_a = [BusA(i, None) for i in range(1, DRIVERS_A_AMOUNT + 1)]
        buses_b = [BusB(i, None) for i in range(
            BUSES_A_AMOUNT + 1, BUSES_INIT_AMOUNT + 1)]
        
        self.distribute_items(
            buses_a, self.rm.add_bus_to_start, self.rm.add_bus_to_end)
        self.distribute_items(
            buses_b, self.rm.add_bus_to_start, self.rm.add_bus_to_end)

    def distribute_items(self, items, add_to_start, add_to_end):
        start_count, _ = self.distribute_evenly_with_remainder(len(items))
        for i, item in enumerate(items):
            if i < start_count:
                add_to_start(item)
            else:
                add_to_end(item)

    def distribute_evenly_with_remainder(self, total_count, part_a_ratio=0.5):
        if total_count % 2 != 0:
            part_a_count = int(total_count * part_a_ratio)
            part_b_count = total_count - part_a_count
        else:
            part_a_count = total_count // 2
            part_b_count = total_count // 2

        return part_a_count, part_b_count

### 5.3. Процессор симуляции

Все последующие классы 5.3.1.-5.3.3. являются составными частями класса "процессор симуляции", который предназначен для прогона симуляции.
В симуляции за каждый сначала обрабатывается логика в зависимости от текущего времени (например, добавляется автобус, если сейчас ни один не идёт), 
затем происходит логика автобусов, а затем - логика водителей. Каждый тик - одна минута. 

##### 5.3.1. Обработчик временных промежутков

Класс, предназначенный для того чтобы реализовать определённую логику в зависимости от текущего времени. Например, добавить активный автобус, если больше никакой не ходит и др.

In [161]:
class TimeSegmentProcessor:
    def __init__(self, timer: Timer, route_manager: RouteManager):
        self.timer = timer
        self.rm = route_manager

    def process_time_segment(self):
        if self.timer.is_currently_night_time():
            self.handle_night_time()
        elif self.timer.is_currently_peak_hour():
            self.handle_peak_time()
        else:
            self.handle_default_time()

    def handle_night_time(self):
        # даже ночью поток транспорта должен быть, при том минимальный (по автобусу на каждую из двух сторон)
        if len(self.rm.active_buses) <= 1:
            direction = self.get_bus_direction_for_night()
            driver = self.get_available_driver_for_night(direction)
            bus = self.get_available_bus_for_night(direction)
            if driver and bus:
                self.activate_driver_and_bus(driver, bus, direction)

    def get_bus_direction_for_night(self):
        if not self.rm.active_buses:
            return ROUTE_DIRECTION_TYPE_FORWARD
        return self.rm.active_buses[0].route_direction

    def get_available_driver_for_night(self, direction):
        depot = self.rm.depot_start if direction == ROUTE_DIRECTION_TYPE_FORWARD else self.rm.depot_end
        return depot.get_available_driver(lambda driver: isinstance(driver, DriverB))

    def get_available_bus_for_night(self, direction):
        depot = self.rm.depot_start if direction == ROUTE_DIRECTION_TYPE_FORWARD else self.rm.depot_end
        return depot.get_available_bus(lambda bus: isinstance(bus, BusB))

    def activate_driver_and_bus(self, driver, bus: Bus, direction):
        depot = self.rm.depot_start if direction == ROUTE_DIRECTION_TYPE_FORWARD else self.rm.depot_end
        depot.add_active_driver(driver)
        depot.add_active_bus(bus)
        bus.driver = driver

    def handle_default_time(self):
        # Если нет пикового времени, можно оставить пустой метод
        pass

    def handle_peak_time(self):
        # Обычная логика обработки пикового времени (например, больше автобусов)
        pass

##### 5.3.2. Обработчик автобусов

Класс, хранящий в себе всю логику того, как проходят маршрут автобусы:
- как останавливаются на остановках (начальных, конечных или обычных);
- как ведут себя в пути;
- какая у них экономическая логика.

In [162]:
class BusHandler:
    def __init__(self, timer: Timer, route_manager: RouteManager, money_manager: MoneyManager):
        self.timer = timer
        self.rm = route_manager
        self.mm = money_manager

    def process_active_buses(self):
        for bus in self.rm.active_buses:
            if bus.is_on_station:
                self.handle_bus_at_station(bus)
            else:
                self.handle_bus_on_route(bus)

            bus.current_work_time += 1
            if bus.current_work_time % 60 == 0:
                self.mm.pay_for_maintenance_of_one_bus(bus)

    def handle_bus_at_station(self, bus: Bus):
        # или он только приехал на эту остановку, или он на ней до сих пор стоит
        if bus.current_mins_stopped is None:
            self.process_arrival_at_station(bus)
        else:
            self.update_stop_duration(bus)
        
        # если он больше не на остановке, проделать необходимые процессы
        self.check_bus_departure()

    def process_arrival_at_station(self, bus: Bus):
        index = bus.last_bus_station_index
        bus_station = self.rm.bus_stations[index]
        bus_station.spawn_passengers(self.timer)

        if not self.rm.is_bus_in_the_end(bus):
            self.process_passenger_exchange(bus, bus_station)
        else:
            self.finalize_bus_route(bus)

    def process_passenger_exchange(self, bus: Bus, bus_station: BusStation):
        if self.rm.is_bus_in_the_beginning(bus):
            bus.try_to_take_passengers(bus_station)
        else:
            bus.drop_off_passengers()
            bus.try_to_take_passengers(bus_station)

        stop_duration = bus_station.get_stop_duration(self.timer)
        if stop_duration == 0:
            bus.is_on_station = False
        else:
            bus.stop_duration = stop_duration
            bus.current_mins_stopped = 1

    def finalize_bus_route(self, bus: Bus):
        bus.drop_off_all_passengers()
        self.mm.earn_income_of_bus(bus)
        
        bus.driver.is_in_the_end = True
        bus.total_ride_passengers = 0
        
        bus.is_on_depot = True
        depot = self.rm.depot_end if bus.route_direction == ROUTE_DIRECTION_TYPE_FORWARD else self.rm.depot_start
        depot.add_bus(bus)
        depot.add_driver(bus.driver)
        
        bus.is_active = False

    def check_bus_departure(self, bus: Bus):
        # если автобус продолжает путь (т.е. если не на конце маршрута), он покидает депо
        if not bus.is_on_station:
            if bus.is_on_depot:
                if bus.route_direction == ROUTE_DIRECTION_TYPE_FORWARD:
                    self.rm.depot_start.remove_bus_if_exists(bus)
                    self.rm.depot_start.remove_active_bus_if_exists(bus)
                    self.rm.depot_start.remove_driver_if_exists(bus.driver)
                    self.rm.depot_start.remove_active_driver_if_exists(
                        bus.driver)
                elif bus.route_direction == ROUTE_DIRECTION_TYPE_BACKWARD:
                    self.rm.depot_end.remove_bus_if_exists(bus)
                    self.rm.depot_end.remove_active_bus_if_exists(bus)
                    self.rm.depot_end.remove_driver_if_exists(bus.driver)
                    self.rm.depot_end.remove_active_driver_if_exists(
                        bus.driver)

                bus.is_on_depot = False

            bus.current_mins_no_station = 0

    def update_stop_duration(self, bus: Bus):
        if bus.current_mins_stopped >= bus.stop_duration:
            bus.stop_duration = None
            bus.current_mins_stopped = None
            bus.is_on_station = False
        else:
            bus.current_mins_stopped += 1

    def handle_bus_on_route(self, bus: Bus):
        bus.current_mins_no_station += 1
        if self.rm.has_bus_arrived_the_station(bus):
            bus.current_mins_no_station = None
            bus.is_on_station = True
            bus.last_bus_station_index += 1 if bus.route_direction == ROUTE_DIRECTION_TYPE_FORWARD else -1

##### 5.3.3. Обработчик водителей

Класс, хранящий в себе логику водителей:
- когда они работают;
- когда оказываются на конечной остановке:
    - уходят на пересменку;
    - уходят на перерыв;
    - уходят на выходной;
    - уходят с работы после отработки часов.

In [163]:
class DriverHandler:
    def __init__(self, timer: Timer, route_manager: RouteManager, money_manager: MoneyManager):
        self.timer = timer
        self.rm = route_manager
        self.mm = money_manager
        
    def process_active_drivers(self):
        for driver in self.rm.active_drivers:
            self.handle_driver(driver)

    def handle_driver(self, driver: Driver):
        driver.current_work_time += 1
        if driver.current_work_time % 60 == 0:
            self.mm.pay_salary_to_one_driver(driver)
        if driver.is_in_the_end:
            self.handle_driver_at_end(driver)

    def handle_driver_at_end(self, driver: Driver):
        if driver.can_finish_work_today:
            driver.is_active = False
            driver.has_worked_out_today = True
            return
                
        if isinstance(driver, DriverA):
            if not driver.has_gone_for_the_break_today and self.timer.is_currently_lunch_time():
                driver.has_gone_for_the_break_today = True  
        
        # Логика для Водителя B
        elif isinstance(driver, DriverB):
            if not driver.has_taken_break_today:
                if self.timer.is_currently_break_time(driver):
                    driver.has_taken_break_today = True  
        
        # Проверка пересменки
        self.handle_shift_change(driver)

        if isinstance(driver, DriverA):
            if not driver.has_gone_for_the_break_today and len(self.rm.active_drivers) > 2 and self.timer.is_currently_lunch_time():
                pass  
        elif isinstance(driver, DriverB):
            pass  

##### 5.3.4. Процессор симуляции

Класс, предназначенный для прогона симуляции

Разделён на три подкласса:
- обработчик временных промежутков;
- обработчик активных автобусов;
- обработчик активных водителей.

In [164]:
class SimulationProcessor:
    def __init__(self, timer: Timer, route_manager: RouteManager, money_manager: MoneyManager, schedule_manager: ScheduleManager):
        self.timer = timer
        self.rm = route_manager
        self.mm = money_manager
        self.sm = schedule_manager
        
        self.time_segment_processor = TimeSegmentProcessor(timer, route_manager)
        self.bus_handler = BusHandler(self.timer, self.rm, self.mm)
        self.driver_handler = DriverHandler(self.timer, self.rm, self.mm)
        
    def run(self):
        self.sm.reset()
        while self.timer.current_value != self.timer.end_value:
            self.time_segment_processor.process_time_segment()
            self.bus_handler.process_active_buses()
            self.driver_handler.process_active_drivers()
            self.timer.update_current_value()

### 6. Ручной прогонщик алгоритма

#### 6.1. Модель ручного ПА

Класс, предназначенный для решения алгоритма "в лоб".

In [165]:
class ManualAR(AlgorithmRunner):
    def __init__(self):
        super().__init__(self.setup_timer(), RouteManager(), MoneyManager(), ScheduleManager())
        self.configurator = ARConfigurator(self.rm)
        self.sp = SimulationProcessor(self.timer, self.rm, self.mm, self.sm)
        self.configure_all()

    def setup_timer(self):
        start_time = get_monday_datetime().replace(
            hour=0, minute=0, second=0, microsecond=0
        )
        end_time = start_time + \
            timedelta(weeks=1, days=6, hours=23, minutes=59)
        return Timer(start_time, end_time)
    
    def configure_all(self):
        self.configurator.configure_drivers()
        self.configurator.configure_buses()
        
    def run(self):
        self.sp.run()


#### 6.2. Реализация ручного ПА

In [166]:
manual_ar = ManualAR()
manual_ar.run()

In [167]:
manual_ar.sm.display_buses_schedule()

In [168]:
manual_ar.sm.display_bus_stations_schedule()

In [169]:
manual_ar.sm.display_drivers_schedule()

In [170]:
manual_ar.sm.display_breaks_schedule()

In [171]:
manual_ar.sm.display_shift_change_schedule()

### 6. Генетический алгоритм


Не успел

In [172]:
# class GeneticAlgorithm(AlgorithmRunner):
#     def __init__(self):
#         pass

# genetic_algorithm = GeneticAlgorithm()
# genetic_algorithm.run()